In [8]:
import sys
sys.path.insert(0,'..')
from pg_connection import pg_connection, sql_types
import pandas as pd

In [2]:
conn = pg_connection(database='ipo_monitoring')

## Get all tables from database

In [3]:
tables_query = """SELECT table_catalog, table_schema, table_name
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA IN ('public')"""

df_tables = pd.read_sql_query(tables_query, conn)
tables = df_tables['table_name'].unique().tolist()

## Finding tables with duplicates

In [5]:
dupe_tables = []
for table in tables:
    df_temp = pd.read_sql_table(table, conn)
    try:
        if len(df_temp.loc[df_temp.duplicated()]) > 0:
            dupe_tables.append(table)
    except Exception as e:
        print(f"error for {table} - {e}")

ref_sources - unhashable type: 'dict'


In [6]:
dupe_tables

['source_shanghai_raw',
 'source_east_money_raw',
 'source_asx_raw',
 'source_tkipo_raw',
 'source_ipohub_raw',
 'source_shanghai',
 'source_east_money',
 'source_tmx_raw']

Now that I have the tables with duplicates, I will read the tables again, drop duplicates and write them back to SQL. However, I have to be careful that the data types don't change. 

### source_shanghai_raw

In [10]:
source_shanghai_raw = pd.read_sql_table('source_shanghai_raw', conn)
print(len(source_shanghai_raw))

source_shanghai_raw.drop_duplicates(inplace=True)
num_cols = len(source_shanghai_raw.columns)
source_shanghai_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_shanghai_raw))
source_shanghai_raw.head()

15565
14


,time_added,time_removed,new_share_name,subscription_date,price,initial_shares_total_shares,deal_size,pe_ratio,online_and_offline_circulation,online_purchase_limit,success_rate,announcement_of_winning_results
0,2022-06-22 20:50:00,NaT,None,None,--,None,None,--,None,None,None,None
5183,2022-06-22 20:50:00,NaT,2022-05-26,136.66,2666.672666.67,364427.1222,127.9,930.951578.4026,0.65,0.04404097,2022-05-30,2022-06-08
5184,2022-06-22 20:50:00,NaT,2022-06-22,41.27,2240.08282240.0828,None,None,725.81155.7653,0.5,0.03347961,2022-06-24,None
5185,2022-06-22 20:50:00,NaT,2022-05-18,15.37,1124311243,172804.91,0,2361.055509.05,1.55,0.0483009,2022-05-20,2022-05-27
5186,2022-06-22 20:50:00,NaT,2022-06-21,14.62,1200012000,None,None,108001200,3.6,0.06706069,2022-06-23,None


In [18]:
source_shanghai_raw.to_sql('source_shanghai_raw', conn, if_exists='replace', index=False)

### source_east_money_raw

In [11]:
source_east_money_raw = pd.read_sql_table('source_east_money_raw', conn)
print(len(source_east_money_raw))

source_east_money_raw.drop_duplicates(inplace=True)
num_cols = len(source_east_money_raw.columns)
source_east_money_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_east_money_raw))
source_east_money_raw.head()

5899
5852


,ticker,company_name,relevant_information,subscription_code,shares_offered_total,shares_offered_online,top_grid_purchase_match_mkt_value,subscription_limit,price,latest_price,...,pe_ratio_industry,success_rate,number_of_inquiries,number_of_allotments,market_segment,percent_change,change_from_ipo_price,document_link,time_added,time_removed
0,1266,宏英智能,详细 研报 股吧,1266,-,734,7.00,0.70,-,-,...,-,-,-,-,待上市,-,-,None,2022-02-08 16:50:00,2022-02-08 21:50:00
1,688223,晶科能源,详细 研报 股吧,787223,200000,42000,280.00,28.00,5.00,10.32,...,48.8,0.149621,1275.26,11283,-,104.00,2600,None,2022-02-08 16:50:00,2022-02-09 01:50:00
2,301116,益客食品,详细 研报 股吧,301116,4490,1626,7.50,0.75,11.40,23.09,...,29.93,0.020086,4775.02,9147,-,203.60,11605,None,2022-02-08 16:50:00,2022-02-09 01:50:00
3,603122,合富中国,详细 研报 股吧,732122,9951,8956,39.00,3.90,4.19,-,...,18.74,0.044919,3255.63,12786,待上市,-,-,None,2022-02-08 16:50:00,2022-02-09 01:50:00
4,688176,亚虹医药,详细 研报 股吧,787176,11000,3157,20.50,2.05,22.98,15.47,...,38.63,0.035244,2875.44,9423,-,-19.06,-2190,None,2022-02-08 16:50:00,2022-02-09 01:50:00


In [19]:
source_east_money_raw.to_sql('source_east_money_raw', conn, if_exists='replace', index=False)

### source_asx_raw

In [12]:
source_asx_raw = pd.read_sql_table('source_asx_raw', conn)
print(len(source_asx_raw))

source_asx_raw.drop_duplicates(inplace=True)
num_cols = len(source_asx_raw.columns)
source_asx_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_asx_raw))
source_asx_raw.head()

910
826


,time_added,time_removed,listing_info,company_name,ipo_date,exchange
0,2021-03-16 22:26:00,NaT,Metal Tiger PLC - TBA,Metal Tiger PLC,NaT,Australian Stock Exchange
1,2021-03-16 22:26:00,NaT,Juno Minerals Limited - TBA,Juno Minerals Limited,NaT,Australian Stock Exchange
2,2021-03-18 10:01:00,NaT,Peregrine Gold Ltd - 22 March 2021 #,Peregrine Gold Ltd,2021-03-22,Australian Stock Exchange
3,2021-03-18 10:01:00,NaT,Tulla Resources Plc - \t18 March 2021 12:00PM ##,Tulla Resources Plc,2021-03-18,Australian Stock Exchange
4,2021-03-18 10:01:00,NaT,Bastion Minerals Limited - 16 March 2021 11:00...,Bastion Minerals Limited,2021-03-16,Australian Stock Exchange


In [20]:
source_asx_raw.to_sql('source_asx_raw', conn, if_exists='replace', index=False)

### source_tkipo_raw

In [13]:
source_tkipo_raw = pd.read_sql_table('source_tkipo_raw', conn)
print(len(source_tkipo_raw))

source_tkipo_raw.drop_duplicates(inplace=True)
num_cols = len(source_tkipo_raw.columns)
source_tkipo_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_tkipo_raw))
source_tkipo_raw.head()

279722
792


,time_added,time_removed,company_name,ipo_date,ticker,shares_outstanding,price_range,price,bookbuilding_period,opening_price,percent_change,underwriters,price_range_expected_date,price_expected_date,exchange
0,2021-03-16 22:26:00,NaT,"Broad-Minded Co.,Ltd.",2021-03-26,7343,"900,000 - 1,035,000",710-810,NaN,03/10 - 03/16,None,None,Mizuho,None,17-Mar,Japan Stock Exchange - MTH
1,2021-03-16 22:26:00,NaT,Terminalcare Support Institute Inc．,2021-03-19,7362,"400,000 - 460,000","1,700-2,000",2000.0,03/03 - 03/09,None,None,Nomura,None,None,Japan Stock Exchange - MTH
2,2021-03-16 22:26:00,NaT,"e-LogiT co.,ltd.",2021-03-26,9327,"915,600 - 1,048,800","1,400-1,500",NaN,03/10 - 03/16,None,None,Ichiyoshi,None,17-Mar,Japan Stock Exchange - JQS
3,2021-03-16 22:26:00,NaT,G-NEXT Inc.,2021-03-25,4179,"556,500 - 639,900","1,130-1,230",NaN,03/10 - 03/16,None,None,SMBC Nikko,None,17-Mar,Japan Stock Exchange - MTH
4,2021-03-16 22:26:00,NaT,"i-plug,Inc.",2021-03-18,4177,"511,600 - 588,300","2,470-2,620",2620.0,03/03 - 03/09,None,None,Daiwa,None,None,Japan Stock Exchange - MTH


In [21]:
source_tkipo_raw.to_sql('source_tkipo_raw', conn, if_exists='replace', index=False)

### source_ipohub_raw

In [14]:
source_ipohub_raw = pd.read_sql_table('source_ipohub_raw', conn)
print(len(source_ipohub_raw))

source_ipohub_raw.drop_duplicates(inplace=True)
num_cols = len(source_ipohub_raw.columns)
source_ipohub_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_ipohub_raw))
source_ipohub_raw.head()

5799408
379


,time_added,time_removed,company_name,exchange,listing_type,subscription_period,price,market_cap,deal_size,status,ipo_date,price_range,currency
0,2021-11-30 15:08:29.729944,NaT,Vici iGaming AB,Spotlight Stock Market (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
1,2021-11-30 15:08:29.729944,NaT,Dala Energi AB,Nasdaq First North GM (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
96463,2021-11-30 15:08:29.729944,NaT,Paydrive AB,NGM Nordic SME (Sweden),IPO,23 Nov - 07 Dec 2021,17.1,100M SEK,20M-23M SEK,Active,NaT,None,SEK
96689,2021-11-30 15:08:29.729944,NaT,Eco-Oil Sverige AB,NGM Nordic SME (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
111841,2021-11-30 15:08:29.729944,NaT,Titania Holding AB,Nasdaq First North GM (Sweden),IPO,30 Nov - 09 Dec 2021,20.0,1B SEK,430M SEK,Active,NaT,None,SEK


In [22]:
source_ipohub_raw.to_sql('source_ipohub_raw', conn, if_exists='replace', index=False)

### source_shanghai

In [15]:
source_shanghai = pd.read_sql_table('source_ipohub_raw', conn)
print(len(source_shanghai))

source_shanghai.drop_duplicates(inplace=True)
num_cols = len(source_shanghai.columns)
source_shanghai.dropna(how='all', thresh=(num_cols-3))

print(len(source_shanghai))
source_shanghai.head()

5799408
379


,time_added,time_removed,company_name,exchange,listing_type,subscription_period,price,market_cap,deal_size,status,ipo_date,price_range,currency
0,2021-11-30 15:08:29.729944,NaT,Vici iGaming AB,Spotlight Stock Market (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
1,2021-11-30 15:08:29.729944,NaT,Dala Energi AB,Nasdaq First North GM (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
96463,2021-11-30 15:08:29.729944,NaT,Paydrive AB,NGM Nordic SME (Sweden),IPO,23 Nov - 07 Dec 2021,17.1,100M SEK,20M-23M SEK,Active,NaT,None,SEK
96689,2021-11-30 15:08:29.729944,NaT,Eco-Oil Sverige AB,NGM Nordic SME (Sweden),Listing,None,NaN,None,None,None,NaT,None,None
111841,2021-11-30 15:08:29.729944,NaT,Titania Holding AB,Nasdaq First North GM (Sweden),IPO,30 Nov - 09 Dec 2021,20.0,1B SEK,430M SEK,Active,NaT,None,SEK


In [23]:
source_shanghai.to_sql('source_shanghai', conn, if_exists='replace', index=False)

### source_tmx_raw

In [17]:
source_tmx_raw = pd.read_sql_table('source_tmx_raw', conn)
print(len(source_tmx_raw))

source_tmx_raw.drop_duplicates(inplace=True)
num_cols = len(source_tmx_raw.columns)
source_tmx_raw.dropna(how='all', thresh=(num_cols-3))

print(len(source_tmx_raw))
source_tmx_raw.head()

12416
12108


,identification,entry_date,cusip,change_type,company_name,security_description,changed_cusip_new,changed_company_name_new,changed_security_description_new,list_symbol,delist_symbol,split/consolidation_factor,effective_date,details,modification_date,exchange,file,time_added,time_removed
0,20210596,2021-06-10,None,Supplemental,Novo Resources Corp.,Warrants,None,None,None,NVO.WT.A,None,None,2021-06-14,None,NaT,TSX,20210614TSXListingsChanges_NC.txt,2021-07-13 17:48:03.245725,2021-07-20 16:33:00
1,20210593,2021-06-10,None,New Listing,MediaValet Inc.,Common Shares,None,None,None,MVP,None,None,2021-06-14,transfer from TSX Venture Exchange,NaT,TSX,20210614TSXListingsChanges_NC.txt,2021-07-13 17:48:03.245725,2021-07-20 16:33:00
2,20210588,2021-06-09,None,New Listing,Blackline Safety Corp.,Common Shares,None,None,None,BLN,None,None,2021-06-11,transfer from TSX Venture Exchange,NaT,TSX,20210614TSXListingsChanges_NC.txt,2021-07-13 17:48:03.245725,2021-07-20 16:33:00
3,20210559,2021-06-01,None,Delist,Global Telecom & Utilities Income Fund,Units,None,None,None,None,HGI.UN,None,2021-06-15,following redemption,NaT,TSX,20210614TSXListingsChanges_NC.txt,2021-07-13 17:48:03.245725,2021-07-20 16:33:00
4,20210598,2021-06-11,None,New Listing,Dynamic Active Emerging Markets ETF,Units,None,None,None,DXEM,None,None,2021-06-15,Initial Public Offering,NaT,TSX,20210614TSXListingsChanges_NC.txt,2021-07-13 17:48:03.245725,2021-07-20 16:33:00


In [24]:
source_tmx_raw.to_sql('source_tmx_raw', conn, if_exists='replace', index=False)

In [25]:
conn.close()